# Projeto Final — IAA 2025/2026  
## Consumo Energético — Clustering e Previsão
## 110942 Luís Lourenço 101018 Miguel Bento
### Estrutura

1. Business Understanding  
2. Data Understanding
3. Data Preparation 
4. Modeling 
5. Evaluation
6. Conclusions


## 1 — Business Understanding

O Município da Maia disponibilizou dados de consumo energético de edifícios institucionais, recolhidos a cada 15 minutos. O objectivo deste projeto é analisar esses consumos de forma a apoiar a gestão energética municipal, recorrendo a técnicas de clustering e previsão.

Em particular, pretende-se:

* identificar grupos de edifícios com padrões de consumo semelhantes;
* caracterizar perfis energéticos típicos e detectar comportamentos atípicos;
* avaliar a previsibilidade do consumo futuro em diferentes tipos de edifícios;
* comparar modelos de previsão simples e avançados com um baseline semanal.

O sucesso do projeto é avaliado pela capacidade de obter clusters interpretáveis e previsões superiores ao baseline, com potencial utilidade para planeamento energético e optimização de custos.


## 2 — Data Understanding

O dataset contém medições de potência activa e reactiva para múltiplos CPEs, ao longo de vários anos, com periodicidade de 15 minutos.

A análise exploratória revelou:

* distribuições fortemente assimétricas, com muitos valores baixos e poucos consumos muito elevados;
* a presença de edifícios com comportamento claramente distinto, incluindo um grande outlier energético;
* padrões diurnos e semanais bem definidos na maioria dos CPEs, com redução do consumo durante a noite e fins de semana;
* correlação moderada entre potência activa e potências reactivas, sugerindo influência do tipo de equipamentos instalados.

A variável **Consumo** encontra-se sem valores no dataset fornecido, pelo que todas as análises e modelos utilizaram exclusivamente a **Potência Ativa** como variável de interesse.

Estes resultados confirmam que os dados possuem estrutura temporal e variabilidade suficiente para clustering e previsão.

### Fig. 1 — Distribuição da Potência Ativa (histograma)

*(Distribuição global da Potência Ativa, evidencia forte assimetria e presença de valores extremos.)*

![](figures/hist_pot_ativa.png)

### Fig. 2 — Perfil médio diário de consumo (24h)

*(Perfil médio horário agregado, evidencia padrões diurnos e períodos de baixa actividade.)*

![](figures/hourly_profile.png)


## 3 — Data Preparation e Feature Engineering

As séries temporais foram preparadas para garantir consistência e comparabilidade entre CPEs.

Os principais passos incluíram:

* reindexação de cada série para uma grelha regular de 15 minutos;
* interpolação linear apenas para falhas curtas (até 1 hora);
* remoção de CPEs com séries demasiado incompletas ou curtas;
* normalização das features para evitar enviesamento por escala.

Após filtragem, permaneceram **84 CPEs válidos**.

Para o clustering e os modelos de previsão foram extraídas features que capturam:

* perfis horários e semanais médios;
* nível médio de consumo e variabilidade;
* autocorrelação diária (lag de 24h);
* indicadores estruturais como base load, picos diários e comportamento reativo.

Este conjunto de features fornece uma representação compacta e informativa do comportamento energético de cada edifício, adequada tanto para segmentação como para modelação preditiva.

## 4 — Clustering

### 4.1 K-Means — Segmentação Global dos Consumidores

O algoritmo K-Means foi aplicado sobre o conjunto de features normalizadas com o objectivo de identificar grupos globais de edifícios com comportamentos energéticos semelhantes.

A escolha do número de clusters baseou-se na análise conjunta dos métodos **Elbow** e **Silhouette**, tendo-se observado que **k = 3** oferece o melhor compromisso entre qualidade de separação e interpretabilidade.

A visualização dos clusters através de **PCA** revelou uma estrutura clara:

* um ponto isolado correspondente a um grande consumidor energético;
* dois grupos principais, diferenciados sobretudo pelo nível médio de consumo e pela variabilidade diária.

### Fig. 3 — Projecção PCA dos clusters (K-Means, k = 3)

*(Visualização dos clusters no espaço PCA, destaca um grande outlier energético e dois grupos principais.)*

![](figures/pca_kmeans_k3.png)


#### Interpretação dos clusters (k = 3)

A caracterização estatística e a análise dos perfis horários e semanais permitem interpretar os clusters da seguinte forma:

* **Cluster 0 — Edifícios de baixo consumo e elevada regularidade**
  Inclui a maioria dos CPEs, com potência média reduzida, baixa variabilidade e elevada autocorrelação diária. Representa edifícios pequenos ou instalações com funcionamento previsível, sendo os mais fáceis de prever.

* **Cluster 1 — Outlier energético (1 CPE)**
  Contém um único edifício com consumo médio e máximo muito superiores aos restantes, elevada variabilidade e comportamento complexo. A validação com dados reais confirma a correspondência com os Paços do Concelho / Torre do Lidador.

* **Cluster 2 — Edifícios intermédios com padrões irregulares**
  Agrupa instalações com consumo moderado a elevado e maior variabilidade diária, como escolas, pavilhões, piscinas e infraestruturas desportivas.

A correspondência entre os clusters obtidos e a tipologia real dos edifícios confirma a validade da segmentação produzida pelo K-Means.

### Fig. 4 — Perfis médios diários por cluster (K-Means)

*(Perfis horários médios de cada cluster, evidencia diferenças claras de regularidade e intensidade.)*

![Perfis diários por cluster](figures/daily_profiles_clusters.png)


### 4.2 DBSCAN — Detecção de Clusters Densos e Outliers

Para complementar a análise, foi aplicado o algoritmo **DBSCAN**, que agrupa pontos com base em densidade local e permite identificar explicitamente outliers.

Após redução de dimensionalidade com PCA, o parâmetro **eps = 0.60** foi seleccionado como configuração final, por produzir uma segmentação estável e interpretável.

O DBSCAN identificou:

* **3 clusters densos**, correspondentes a perfis energéticos bem definidos;
* **59 outliers**, representando edifícios cujo comportamento não forma grupos suficientemente compactos.

Este resultado evidencia a elevada heterogeneidade dos dados e reforça a existência de instalações com comportamento energético atípico.

### Fig. 5 — Clusters e outliers identificados pelo DBSCAN (PCA)

*(Visualização dos clusters densos e dos outliers identificados pelo DBSCAN no espaço PCA.)*

![PCA DBSCAN](figures/pca_dbscan.png)


### 4.3 Comparação entre K-Means e DBSCAN

A comparação entre os dois algoritmos revela diferenças fundamentais:

* O **K-Means** força a atribuição de todos os CPEs a um cluster, sendo adequado para uma segmentação global do parque edifício.
* O **DBSCAN** é mais conservador, identificando apenas grupos densos e classificando uma parte significativa dos edifícios como outliers.

A matriz de contingência entre os rótulos dos dois métodos mostra baixa correspondência directa, o que é esperado dado que:

* o K-Means assume clusters esféricos e homogéneos;
* o DBSCAN depende da densidade local dos dados.

### Fig. 6 — Matriz de contingência: K-Means vs DBSCAN

*(Correspondência entre os clusters obtidos pelos dois algoritmos.)*

![Matriz KMeans vs DBSCAN](figures/contingency_kmeans_dbscan.png)


Apesar disso, ambos os métodos concordam na identificação do grande outlier energético e na existência de grupos com comportamento regular e previsível.

Assim, os algoritmos devem ser vistos como **complementares**:

* o K-Means fornece uma segmentação global e interpretável;
* o DBSCAN destaca padrões densos inequívocos e edifícios atípicos.

Enquanto o K-Means impõe uma segmentação global,o que força a todos os edifícios a pertencer a um grupo, o DBSCAN adopta uma abordagem conservadora baseada em densidade local. Num dataset energético altamente heterogéneo, esta diferença conceptual conduz a resultados distintos: o K-Means revela a estrutura global do parque edifício, enquanto o DBSCAN identifica apenas padrões inequívocos e destaca explicitamente a maioria dos edifícios como comportamentos atípicos.


### 4.4 Conclusão do Clustering

O clustering revelou que os edifícios municipais apresentam perfis energéticos claramente distintos, com diferenças significativas ao nível do consumo médio, variabilidade e regularidade temporal.

A segmentação obtida permite:

* caracterizar diferentes tipos de consumidores;
* identificar edifícios estáveis e edifícios problemáticos;
* apoiar a análise da previsibilidade do consumo energético.

Estes resultados são utilizados directamente na secção seguinte para interpretar o desempenho dos modelos de previsão por cluster.


## 5 — Previsão do Consumo Energético

O objectivo desta secção é avaliar a capacidade de previsão do consumo energético dos edifícios municipais, comparando diferentes abordagens: um baseline simples, um modelo clássico de séries temporais e modelos baseados em *feature sets*.

A avaliação foi realizada com uma separação **estritamente temporal**, utilizando os **70% iniciais de cada série para treino** e os **30% finais para teste**, garantindo ausência de *data leakage*.

### 5.1 Baseline Semanal

### Fig. 7 — Real vs Baseline semanal (CPE mais central do Cluster 0)

*(Exemplo de edifício altamente regular, onde o baseline apresenta desempenho quase perfeito.)*

![Baseline Cluster 0](figures/baseline_cluster0.png)


O baseline utilizado assume que o consumo num determinado instante é igual ao consumo observado no mesmo instante uma semana antes.

Os resultados mostram que:

* o baseline apresenta desempenho quase perfeito em edifícios com padrões altamente regulares;
* os erros aumentam significativamente em instalações com maior variabilidade e picos de consumo;
* o erro varia fortemente entre clusters, sendo mínimo no Cluster 0 e máximo no outlier energético (Cluster 1).

Este comportamento confirma que a repetição semanal é uma forte referência, mas insuficiente para perfis mais complexos.

### 5.2 ARIMA — Modelo Clássico de Séries Temporais

### Fig. 8 — Real vs ARIMA (CPE mais central do Cluster 2)

*(Exemplo típico de previsão ARIMA, evidencia suavização excessiva e incapacidade de captar padrões diários.)*

![ARIMA Cluster 2](figures/arima_cluster2.png)

Foi aplicado um modelo **ARIMA univariado**, treinado individualmente por CPE, utilizando apenas a série de potência activa.

O desempenho do ARIMA foi consistentemente **inferior ao baseline semanal**, tanto em MAE como em RMSE. As previsões apresentaram forte suavização e convergência para valores médios, refletindo a incapacidade do modelo em capturar padrões diários e semanais dominantes.

Este resultado era esperado, dado que:

* o modelo é estritamente univariado;
* não inclui componentes sazonais;
* não utiliza variáveis exógenas.

Apesar do desempenho fraco, o ARIMA cumpre o seu papel como **baseline estatístico clássico**, evidenciando as limitações de abordagens tradicionais neste contexto energético.

O ARIMA serve assim como um controlo experimental: demonstra que, mesmo em séries com elevada autocorrelação, modelos que ignoram estrutura sazonal explícita e informação contextual são inadequados para previsão energética realista.

### 5.3 Modelos Baseados em Features

#### Random Forest

A Random Forest apresentou uma melhoria muito significativa face ao baseline, reduzindo o erro absoluto médio (MAE) em cerca de **60%**. O modelo capturou eficazmente padrões de curto prazo e dinâmicas intra-diárias.

No entanto:

* o RMSE manteve-se elevado em edifícios com picos abruptos;
* nenhum dos modelos conseguiu antecipar totalmente eventos extremos, devido à ausência de variáveis exógenas.

A análise por cluster mostrou que:

* o Cluster 0 é facilmente previsível;
* o Cluster 2 apresenta dificuldade intermédia;
* o Cluster 1 continua a ser extremamente difícil de prever, embora a RF reduza substancialmente o erro face ao baseline.

#### XGBoost

O modelo XGBoost foi progressivamente optimizado, resultando numa versão final que reduz a dependência de autocorrelações triviais e explora padrões intra-horários, diários e semanais.

O **XGBoost BEST** apresentou:

* o **menor MAE global** entre todos os modelos testados;
* desempenho particularmente forte em edifícios com variabilidade moderada;
* maior sensibilidade a picos extremos, refletindo-se num RMSE ligeiramente superior ao da Random Forest.

### 5.4 Comparação Global dos Modelos

| Modelo         | MAE   | RMSE  | SMAPE (%) | R²    |
| -------------- | ----- | ----- | --------- | ----- |
| Baseline (t−7) | 2.187 | 7.683 | 39.62     | 0.880 |
| ARIMA          | 5.177 | 6.648 | 87.52     | -0.278|
| Random Forest  | 0.800 | 2.487 | 43.15     | 0.988 |
| XGBoost (BEST) | 0.624 | 3.050 | 42.59     | 0.982 |

A comparação entre os modelos revela que:

* o **baseline semanal** é adequado apenas para edifícios altamente regulares;
* o **ARIMA** é claramente superado por todas as restantes abordagens;
* a **Random Forest** apresenta maior robustez face a picos, resultando no melhor RMSE global;
* o **XGBoost BEST** oferece a melhor precisão média (MAE), sendo o modelo mais eficaz na maioria dos edifícios.

Os resultados mostram que não existe um modelo universalmente superior. A Random Forest apresenta maior robustez a valores extremos, enquanto o XGBoost optimizado oferece maior precisão média. Esta diferença está directamente ligada à natureza dos perfis energéticos identificados no clustering.

### 5.5 Análise Visual — Real vs Previsto

### Fig. 9 — Comparação Real vs Previsto (XGBoost BEST,CPE mais central do Cluster 2)

*(Comparação entre valores reais e previstos pelo XGBoost BEST, evidenciando melhor captura da dinâmica temporal.)*

![](figures/xgb_cluster2.png)


As visualizações *Real vs Previsto*, realizadas para um CPE representativo de cada cluster, confirmam de forma clara as conclusões quantitativas:

* em edifícios estáveis, todos os modelos produzem previsões próximas da realidade;
* em perfis intermédios, os modelos baseados em features capturam melhor a dinâmica do consumo;
* no outlier energético, todos os modelos enfrentam dificuldades, embora RF e XGBoost superem claramente o baseline e o ARIMA.

Estas visualizações evidenciam que a **dificuldade de previsão está fortemente ligada ao cluster energético** identificado na fase de clustering.

### 5.6 Síntese da Previsão

Em síntese:

* a previsibilidade do consumo varia significativamente entre edifícios;
* modelos baseados em features oferecem ganhos substanciais face a abordagens clássicas;
* a ausência de variáveis exógenas limita a previsão de picos abruptos;
* a segmentação por clusters é fundamental para interpretar correctamente os erros de previsão.

## 6. Conclusões
### 6.1 Síntese do trabalho realizado

Neste trabalho foi aplicado o processo CRISP-DM ao conjunto de dados de consumo energético da Câmara Municipal da Maia, com dois objectivos principais:
- Caracterizar os diferentes consumidores através de técnicas de clustering 
- Avaliar a capacidade de previsão do consumo energético utilizando diferentes abordagens de modelação.

O estudo combinou análise exploratória, extração de features, aprendizagem não supervisionada e modelos de previsão, permitindo obter uma visão integrada dos padrões de consumo e da sua previsibilidade.

### 6.2 Conclusões do Clustering

A aplicação de técnicas de clustering permitiu identificar perfis energéticos distintos entre os CPEs analisados.

- O K-Means revelou-se eficaz na separação global dos consumidores, produzindo clusters interpretáveis e coerentes com diferentes níveis de consumo, variabilidade e regularidade.

- O DBSCAN, por sua vez, identificou apenas padrões densos e inequívocos, classificando muitos CPEs como outliers, o que evidencia a heterogeneidade dos comportamentos energéticos.

A comparação entre ambos mostrou que os métodos são complementares:
o K-Means é adequado para uma segmentação global, enquanto o DBSCAN é útil para detecção de perfis muito específicos ou atípicos.

A validação com edifícios reais confirmou que os clusters obtidos correspondem a tipologias plausíveis de consumo (edifícios estáveis, perfis intermédios e grandes consumidores/outliers).

### 6.3 Conclusões da Previsão

A avaliação dos modelos de previsão evidenciou diferenças claras entre abordagens:

- O baseline semanal mostrou-se surpreendentemente forte para edifícios com comportamento estável, confirmando que grande parte do consumo apresenta elevada repetibilidade semanal.

- O ARIMA univariado, sem componente sazonal, apresentou previsões praticamente constantes, demonstrando limitações claras na captura de padrões diários e semanais característicos do consumo energético.

- Os modelos baseados em feature sets (Random Forest e XGBoost) superaram consistentemente o baseline e o ARIMA, capturando melhor a dinâmica temporal e a variabilidade dos dados.

- O XGBoost, após afinação e selecção de features, apresentou o melhor desempenho global, especialmente em edifícios com maior variabilidade e picos de consumo.

As visualizações de Real vs Previsto mostraram de forma clara que a dificuldade de previsão varia significativamente entre clusters, sendo maior em perfis mais irregulares e com eventos extremos.

### 6.4 Considerações metodológicas

A separação treino–teste foi realizada de forma estritamente temporal e individual por CPE, utilizando os 70% iniciais de cada série para treino e os 30% finais para teste. Esta abordagem garante ausência de data leakage e avalia correctamente a capacidade de previsão do consumo futuro de edifícios já observados.

As features foram construídas apenas com informação passada ou com características estruturais dos consumidores, assegurando consistência metodológica.

### 6.5 Limitações e trabalho futuro

Apesar dos bons resultados obtidos, existem limitações relevantes:

- A ausência de variáveis exógenas (meteorologia, ocupação, eventos) limita a capacidade de previsão de picos abruptos.

- Os modelos avaliados não foram optimizados para previsão multi-horizonte de longo prazo.

- O ARIMA poderia ser melhorado com componentes sazonais explícitas, embora tal não fosse o foco deste trabalho.

Como trabalho futuro, seria interessante integrar dados externos, explorar modelos híbridos e avaliar estratégias de previsão específicas por cluster.

### 6.6 Conclusão final

Em síntese, este trabalho demonstra que:

- Os dados de consumo energético apresentam estrutura suficiente para segmentação significativa;

- A previsibilidade depende fortemente do perfil energético do consumidor;

- Modelos baseados em features oferecem ganhos substanciais face a abordagens clássicas univariadas.

Os resultados confirmam o potencial deste tipo de dados para apoiar análise energética e sistemas de previsão em contexto real.

In [7]:
import json

with open("IAA_relatorio_markdown.ipynb", "r", encoding="utf-8") as f:
    nb = json.load(f)

for i, cell in enumerate(nb["cells"]):
    if cell["cell_type"] == "code":
        print(f"\n# ----- CODE CELL {i} ----- #\n")
        print("".join(cell["source"]))
    else:
        print(f"\n# ----- MARKDOWN CELL {i} ----- #\n")
        print("".join(cell["source"]))


# ----- MARKDOWN CELL 0 ----- #

# Projeto Final — IAA 2025/2026  
## Consumo Energético — Clustering e Previsão
## 110942 Luís Lourenço 101018 Miguel Bento
### Estrutura

1. Business Understanding  
2. Data Understanding
3. Data Preparation 
4. Modeling 
5. Evaluation
6. Conclusions


# ----- MARKDOWN CELL 1 ----- #

## 1 — Business Understanding

O Município da Maia disponibilizou dados de consumo energético de edifícios institucionais, recolhidos a cada 15 minutos. O objectivo deste projeto é analisar esses consumos de forma a apoiar a gestão energética municipal, recorrendo a técnicas de clustering e previsão.

Em particular, pretende-se:

* identificar grupos de edifícios com padrões de consumo semelhantes;
* caracterizar perfis energéticos típicos e detectar comportamentos atípicos;
* avaliar a previsibilidade do consumo futuro em diferentes tipos de edifícios;
* comparar modelos de previsão simples e avançados com um baseline semanal.

O sucesso do projeto é avaliado pela capac